## Import Libraries

In [64]:
#Install libraries if using google colab
#pip install pandapower
#pip install simbench

In [10]:
#Import the pandapower and the networks module:
import pandapower as pp
import pandapower.networks as nw
import simbench as sb
import pandas as pd
import numpy as np
import os, sys, time
'''
#Plotting
from pandapower.plotting.plotly.mapbox_plot import set_mapbox_token
from pandapower.plotting.plotly import simple_plotly, pf_res_plotly, vlevel_plotly
import matplotlib.pyplot as plt
import pandapower.plotting as plot
import pandapower.plotting.plotly as pplotly
try:
    import seaborn
    colors = seaborn.color_palette()
except:
    colors = ["b", "g", "r", "c", "y"]
%matplotlib inline
'''
#Timeseries
import tempfile
from pandapower.control import ConstControl
from pandapower.timeseries import DFData
from pandapower.timeseries import OutputWriter
from pandapower.timeseries.run_time_series import run_timeseries

## Define Functions

In [24]:
def init_net():
    '''
    Drops any added load/generation
    Initialization of load and generation p_mw and q_mvar is needed because the run_timeseries replaces them after every iteration
    Drops Constcontrol objects created by sb.apply_cost_controllers
    
    OUTPUT - 
        net- Pandapower network with initial values
    '''
    net.load=net.load.head(init_cap_l[0])
    net.sgen=net.sgen.head(init_cap_l[1])
    net.load[['p_mw','q_mvar']]=initload
    net.sgen[['p_mw','q_mvar']]=initsgen
    net.controller=net.controller.iloc[0:0]
    return net

def define_log():
    '''
    Creates output writer object required for timeseries simulation
    The timeseries module only calculates the values of variables mentioned here for each simulation. 
    The temporary data gets stored in the output_dir directory
    
    OUTPUT
        ow - Output writer object
    '''
    ow = OutputWriter(net, time_steps, output_path=output_dir, output_file_type=".json")    
    ow.log_variable('res_bus','vm_pu')
    ow.log_variable('res_line', 'loading_percent')
    ow.log_variable('res_trafo', 'loading_percent')
    return ow

def add_loadgen(net_t, loadorgen, conn_at_bus, size_p,size_q,prof):
    '''
    Adds a load or generation to the net.load or net.sgen table. 
    Adds profile name to the profiles variable of the newly addded capacity.

    INPUT
        net_t (PP net) - Pandapower net
        loadorgen (str) - 'sgen' or 'load' for generation or load for additional capacity connected
        conn_at_bus (int) - Bus at which additional capacity is connected
        prof (str) - Name of the profile. Must be available in the net.profiles of the input grid
        
    OUTPUT
        net_t (PP net) - Updated Pandapower net
        
    '''
    if loadorgen=="load":
        pp.create_load(net_t, conn_at_bus, p_mw=size_p, q_mvar=size_q)
        net_t.load.tail(1).profile=prof
    elif loadorgen=="sgen":
        pp.create_sgen(net_t, conn_at_bus, p_mw=size_p, q_mvar=size_q)
        net_t.sgen.tail(1).profile=prof
    else:
        return 0
    return net_t

def load_files():
    '''
    Loads files of previous TS simulation
    
    OUTPUT
        vm_pu,line_load,trafo_load (tuple) - Previous results of timeseries
    '''
    vm_pu_file = os.path.join(output_dir, "res_bus", "vm_pu.json")
    vm_pu = pd.read_json(vm_pu_file)
    line_load_file = os.path.join(output_dir, "res_line", "loading_percent.json")
    line_load = pd.read_json(line_load_file)
    trafo_load_file = os.path.join(output_dir, "res_trafo", "loading_percent.json")
    trafo_load = pd.read_json(trafo_load_file)
    return vm_pu,line_load,trafo_load

def violations_long(net):
    '''
    Checks for any violations created in the grid by additional capacity and returns tuple with the details
    Loads the files created by timeseries simulation. Compares simulation values against the limits mentioned in the input grid.
    
    INPUT
        net (PP net) - Pandapower net
        
    OUTPUT
        check (bool) - tuple of violations with details
        
    '''
    [vm_pu,line_load,trafo_load]=load_files()

    pf_vm_extremes=pd.DataFrame(vm_pu.max())
    pf_vm_extremes.columns=['pf_max_vm_pu']
    pf_vm_extremes['pf_min_vm_pu']=vm_pu.min()
    vm_pu_check = net.bus[['name','vn_kv','min_vm_pu','max_vm_pu']].join( pf_vm_extremes)
    vm_pu_check = vm_pu_check[(vm_pu_check.pf_max_vm_pu>vm_pu_check.max_vm_pu) | (vm_pu_check.pf_min_vm_pu<vm_pu_check.min_vm_pu)] 

    pf_line_extremes=pd.DataFrame(line_load.max())
    pf_line_extremes.columns=['pf_max_loading_percent']
    line_load_check = net.line[['name','from_bus','to_bus','max_loading_percent']].join( pf_line_extremes)
    line_load_check = line_load_check[(line_load_check.pf_max_loading_percent>line_load_check.max_loading_percent)] 

    pf_trafo_extremes=pd.DataFrame(trafo_load.max())
    pf_trafo_extremes.columns=['pf_max_loading_percent']
    trafo_load_check = net.trafo[['name','sn_mva','max_loading_percent']].join( pf_trafo_extremes)
    trafo_load_check = trafo_load_check[(trafo_load_check.pf_max_loading_percent>trafo_load_check.max_loading_percent)] 
    
    return  vm_pu_check,line_load_check, trafo_load_check

def violations(net):
    '''
    Checks for any violations created in the grid by additional capacity.
    Loads the files created by timeseries simulation. Compares simulation values against the limits mentioned in the input grid.
    
    TODO: Create separate violations function for individual input. (Already done, might have to include it here)
    
    INPUT
        net (PP net) - Pandapower net
        
    OUTPUT
        check (bool) - 'True' for no violations. 'False' for violations present
        
    '''
    [vm_pu,line_load,trafo_load]=load_files()

    check = any(np.where(vm_pu.max() > net.bus['max_vm_pu'],True, False))
    check = check or any(np.where(vm_pu.min() < net.bus['min_vm_pu'],True, False))
    check = check or any(np.where(line_load.max() > net.line['max_loading_percent'],True, False))
    check = check or any(np.where(trafo_load.max() > net.trafo['max_loading_percent'],True, False))
    return not check


def feas_chk(net,ow,conn_at_bus,loadorgen, size_p, size_q, prof):
    '''
    Initializes the PPnet, 
    Adds additional capacity, 
    applies load/generation profiles on all the grid elements,
    runs timeseries for the specific case and save the results in the temporary output directory,
    Checks for violations

    BUG: More like pending TODO. Doesnt work for loads. Need to check process of how profiles from simbench are actually 
    getting applied to Constcontrol know the fix. Also will lead to finding how profiles from input will be applied on 
    the input grid.
    TODO: suppress/workaround printing of individual progress bars
    
    INPUT
        net (PP net) - Pandapower net
        ow (Object) - Output writer object
        loadorgen (str) - 'sgen' or 'load' for generation or load for additional capacity connected
        conn_at_bus (int) - Bus at which additional capacity is connected
        size_p (int) - Size of active power of additional capacity
        size_q (int) - Size of reactive power of additional capacity
        prof (str) - Name of the profile. Must be available in the net.profiles of the input grid
        
    OUTPUT
        feas_result (bool) - 'True' for feasible, 'False' for not feasible
        
    '''
    net=init_net()
    net=add_loadgen(net, loadorgen, conn_at_bus, size_p,size_q, prof)
    profiles = sb.get_absolute_values(net, profiles_instead_of_study_cases=True)
    sb.apply_const_controllers(net, profiles)    #create timeseries data from profiles and run powerflow
    run_timeseries(net,time_steps,continue_on_divergence=True,verbose=True)               #Run powerflow only over time_steps
    feas_result=violations(net)
    return feas_result

def max_cap(net,ow,conn_at_bus,loadorgen,ul_p,ll_p,prof):
    '''
    Seach algorithm using feas_chk function over the range of ll_p and ul_p capacities
    
    TODO: Speed up, Try changing ul_p and ll_p as per voltage levels
    
    INPUT
        net (PP net) - Pandapower net
        ow (Object) - Output writer object
        loadorgen (str) - 'sgen' or 'load' for generation or load for additional capacity connected
        conn_at_bus (int) - Bus at which additional capacity is connected
        ll_p (int) - Size of maximum power limit of additional capacity that can be added
        ul_p (int) - Size of minimum additional capacity that can be added (Set as 0)
        prof (str) - Name of the profile. Must be available in the net.profiles of the input grid
        
    OUTPUT
         (int) - Maximum capacitiy of load/generation that can be added at given bus
        
    '''
    no_iter=0
    [ul_chk,mid_chk,ll_chk]=False,False,False
    while not( ((ul_p-ll_p)<s_tol) | (ul_chk & mid_chk) | (no_iter>7) ):
        no_iter=no_iter+1
        mid_p=(ul_p+ll_p)/2
        ul_chk=feas_chk(net,ow,conn_at_bus,loadorgen, size_p=ul_p, size_q=inp_q,prof=prof)
        mid_chk=feas_chk(net,ow,conn_at_bus,loadorgen, size_p=mid_p, size_q=inp_q,prof=prof)
        if mid_chk==True:
            ll_p=mid_p
        elif mid_chk==False:
            ul_p=mid_p
        elif ul_chk:
            return ul_p
            break
    return ll_p
'''
def printProgressBar(iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    """
    https://stackoverflow.com/questions/3173320/text-progress-bar-in-the-console
    
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()
'''
def printProgressBar(iteration, total, prefix='', suffix='', decimals=1, length=100, fill='█'):
    """
    Call in a loop to create terminal progress bar.
    the code is mentioned in : https://stackoverflow.com/questions/3173320/text-progress-bar-in-the-console
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filled_length = int(length * iteration // total)
    bar = fill * filled_length + '-' * (length - filled_length)
    #logger.info('\r%s |%s| %s%% %s' % (prefix, bar, percent, suffix))
    print('\r%s |%s| %s%% %s' % (prefix, bar, percent, suffix), end="")
    # Print New Line on Complete
    if iteration == total:
        print("\n")

def all_cap_map(net,ow,loadorgen,ul_p,ll_p,prof):
    '''
    Iteratre the max_cap function over all busses in the grid. 
    
    TODO: Add progess bar
    
    INPUT
        net (PP net) - Pandapower net
        ow (Object) - Output writer object
        loadorgen (str) - 'sgen' or 'load' for generation or load for additional capacity connected
        ll_p (int) - Size of maximum power limit of additional capacity that can be added
        ul_p (int) - Size of minimum additional capacity that can be added (Set as 0)
        prof (str) - Name of the profile. Must be available in the net.profiles of the input grid
        
    OUTPUT
         allcap (dataframe) - Maximum capacitiy of load/generation that can be added at all buses
    
    '''
    len_items=len(net.bus)
    items = list(range(0, len_items))
    printProgressBar(0, len_items, prefix = 'Progress:', suffix = 'Complete', length = 50)
    allcap=net.bus[['name','vn_kv']]
    allcap['max_add_cap']=np.nan
    for i,conn_at_bus in enumerate(items):
        allcap['max_add_cap'][conn_at_bus]=max_cap(net,ow=ow,conn_at_bus=conn_at_bus,loadorgen=loadorgen,ul_p=ul_p,ll_p=ll_p,prof=prof)
        printProgressBar(i + 1, len_items, prefix = 'Progress:', suffix = 'Complete', length = 50)
    return allcap

def sing_res(net,ow,conn_at_bus,loadorgen, size_p, size_q, prof):
    '''
    Same as feas_chk. Only difference is that instead of returning bool it returns tuple with results
    Initializes the PPnet, 
    Adds additional capacity, 
    applies load/generation profiles on all the grid elements,
    runs timeseries for the specific case and save the results in the temporary output directory,
    Checks for violations

    BUG: More like pending to do. Doesnt work for loads. Need to check process of how profiles from simbench are actually 
    getting applied to Constcontrol know the fix. Also will lead to finding how profiles from input will be applied on 
    the input grid.
    TODO: suppress/workaround printing of individual progress bars
    
    INPUT
        net (PP net) - Pandapower net
        ow (Object) - Output writer object
        loadorgen (str) - 'sgen' or 'load' for generation or load for additional capacity connected
        conn_at_bus (int) - Bus at which additional capacity is connected
        size_p (int) - Size of active power of additional capacity
        size_q (int) - Size of reactive power of additional capacity
        prof (str) - Name of the profile. Must be available in the net.profiles of the input grid
        
    OUTPUT
        result (tuple) - violations details
        
    '''
    net=init_net()
    net=add_loadgen(net, loadorgen, conn_at_bus, size_p,size_q, prof)
    profiles = sb.get_absolute_values(net, profiles_instead_of_study_cases=True)
    sb.apply_const_controllers(net, profiles)    #create timeseries data from profiles and run powerflow
    run_timeseries(net,time_steps,continue_on_divergence=True,verbose=True)               #Run powerflow only over time_steps
    result=violations_long(net)
    return result



## Set Parameters

### Input Grid

In [12]:
'''
Input grid for developing code is sb_code1
sb_code2 is the larger grid to test the code
Initial values of the loads and generation are stored
Maximum voltage limits are relaxed for testing sample code since the limit gets violated for the test grid without adding any capacity
'''
sb_code1 = "1-MV-rural--1-sw"  # rural MV grid of scenario 0 with full switchs
sb_code2 = "1-HVMV-urban-all-0-sw"  # urban hv grid with one connected mv grid which has the subnet 2.202
net = sb.get_simbench_net(sb_code1)
net.bus.max_vm_pu=net.bus.max_vm_pu*1.05
initload=net.load[['p_mw','q_mvar']]
initsgen=net.sgen[['p_mw','q_mvar']]
init_cap_l=[len(net.load) , len(net.sgen)]

### Other parameters

In [13]:
'''
time_steps:Set time steps in range for the timeseries module to compute over. 
    This parameter must be of same length as the length of profiles.
ll_p and ul_p : limits for maximum and minimum capacity that can be added to any bus
inp_q: input reactive power for added capacity. Assumed constant
tol: Search algorithm tolerance (in MW)
output_dir : Set directory for storing the logged varaiables. 
    Commented output_dir line is for setting directory in the temporary files of the computer.
ow: Create the output writer object ow
'''
time_steps=range(96)
ll_p=0
ul_p=90
inp_q=0.1
s_tol=0.005

#output_dir = os.path.join(tempfile.gettempdir(), "simp_cap_v3")
output_dir = os.path.join('C:\\Users\\nitbh\\OneDrive\\Documents\\IIPNB', "simp_cap_v3")
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
ow=define_log()   #For logging variables


## Set Input

In [14]:
'''
Sample input. Use as needed
prof and loadorgen are needed for getting the map
Others are needed in case of individual capacity checks
'''
#size_pmw=10
#size_qmw=0.05
loadorgen='sgen'
#prof='L0-A'
prof='WP4'
#conn_at_bus=2

## Get Map

All_cap_map takes lot of time to calculate capacities of an entire map. Try using inner functions one at a time

In [15]:
#sgen_allcap=all_cap_map(net,ow=ow, loadorgen='sgen', ul_p=ul_p, ll_p=ll_p, prof='WP4')

In [16]:
max_cap(net,ow=ow,conn_at_bus=92, loadorgen='sgen',ul_p=ul_p, ll_p=ll_p, prof='WP4')

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |█████████████████████████████

3.8671875

In [48]:
max_cap(net,ow=ow,conn_at_bus=95, loadorgen='load',ul_p=ul_p, ll_p=ll_p, prof='L0-A')

CalculationNotConverged at time step 0
CalculationNotConverged at time step 1


Progress: |█-------------------------------------------------| 2.1% Complete

CalculationNotConverged at time step 2


Progress: |██------------------------------------------------| 4.2% Complete

CalculationNotConverged at time step 3


...

CalculationNotConverged at time step 84


Progress: |████████████████████████████████████████████------| 89.6% Complete

CalculationNotConverged at time step 85


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete



2.109375

In [26]:
sing_res(net,ow=ow,conn_at_bus=95, loadorgen='load',size_p=10,size_q=0.1, prof='L0-A')

Progress: |██████████████████████████████████████████████████| 100.0% Complete



(              name  vn_kv  min_vm_pu  max_vm_pu  pf_max_vm_pu  pf_min_vm_pu
 88  MV1.101 Bus 88   20.0      0.965    1.10775      1.015018      0.944271
 89  MV1.101 Bus 89   20.0      0.965    1.10775      1.005356      0.885170
 90  MV1.101 Bus 90   20.0      0.965    1.10775      1.000180      0.853495
 91  MV1.101 Bus 91   20.0      0.965    1.10775      0.987642      0.774219
 92  MV1.101 Bus 92   20.0      0.965    1.10775      0.985462      0.760442
 93  MV1.101 Bus 93   20.0      0.965    1.10775      0.984385      0.753593
 94  MV1.101 Bus 94   20.0      0.965    1.10775      0.983611      0.748628
 95  MV1.101 Bus 95   20.0      0.965    1.10775      0.982964      0.744433
 96  MV1.101 Bus 96   20.0      0.965    1.10775      0.982952      0.744408,
                name  from_bus  to_bus  max_loading_percent  \
 74  MV1.101 Line 75         3      78                100.0   
 75  MV1.101 Line 76        78      79                100.0   
 76  MV1.101 Line 77        79      80  

In [52]:
net.profiles

{'load':        G0-A_qload  G0-A_pload  G0-M_qload  G0-M_pload  G3-A_qload  G3-A_pload  \
 0        0.282542    0.143882    0.256264    0.275641    0.241342    0.351171   
 1        0.389736    0.203032    0.473602    0.283333    0.160782    0.349565   
 2        0.402408    0.213184    0.376287    0.248718    0.232880    0.401338   
 3        0.329803    0.170953    0.434676    0.269231    0.135395    0.387960   
 4        0.385284    0.191256    0.437920    0.271795    0.177707    0.387960   
 ...           ...         ...         ...         ...         ...         ...   
 35131    0.321241    0.157417    0.188143    0.250000    0.224418    0.359599   
 35132    0.286994    0.150650    0.210850    0.264103    0.309040    0.409766   
 35133    0.329803    0.169193    0.262752    0.276923    0.266729    0.366288   
 35134    0.235623    0.125203    0.181656    0.248718    0.258267    0.311037   
 35135    0.278432    0.142122    0.236801    0.270513    0.330026    0.409766   
 
      

In [54]:
profiles = sb.get_absolute_values(net, profiles_instead_of_study_cases=True)
profiles

{('load',
  'p_mw'):              0         1         2         3         4         5         6   \
 0      0.028393  0.005514  0.005514  0.005514  0.005514  0.003909  0.003909   
 1      0.028263  0.007749  0.007749  0.007749  0.007749  0.002847  0.002847   
 2      0.032449  0.008986  0.008986  0.008986  0.008986  0.003412  0.003412   
 3      0.031367  0.008970  0.008970  0.008970  0.008970  0.002889  0.002889   
 4      0.031367  0.009169  0.009169  0.009169  0.009169  0.003107  0.003107   
 ...         ...       ...       ...       ...       ...       ...       ...   
 35131  0.029074  0.007307  0.007307  0.007307  0.007307  0.002559  0.002559   
 35132  0.033130  0.006881  0.006881  0.006881  0.006881  0.002519  0.002519   
 35133  0.029615  0.005148  0.005148  0.005148  0.005148  0.002435  0.002435   
 35134  0.025148  0.006037  0.006037  0.006037  0.006037  0.002390  0.002390   
 35135  0.033130  0.006121  0.006121  0.006121  0.006121  0.002544  0.002544   
 
              7   

In [10]:
net.load

,name,bus,p_mw,q_mvar,const_z_percent,const_i_percent,sn_mva,scaling,in_service,type,max_p_mw,max_q_mvar,voltLvl,profile,min_q_mvar,subnet,min_p_mw
0,HV1_MV1.101_load,2,4.950124e-14,2.083897e-20,0.0,0.0,0.2463,1.0,True,NaN,NaN,NaN,5,G3-A,NaN,MV1.101,NaN
1,MV1.101 Load 2,4,2.986680e-17,1.719415e-13,0.0,0.0,0.0880,1.0,True,NaN,NaN,NaN,5,lv_rural1,NaN,MV1.101_LV1.101_eq,NaN
2,MV1.101 Load 3,5,2.986680e-17,1.719415e-13,0.0,0.0,0.0880,1.0,True,NaN,NaN,NaN,5,lv_rural1,NaN,MV1.101_LV1.102_eq,NaN
3,MV1.101 Load 4,6,2.986680e-17,1.719415e-13,0.0,0.0,0.0880,1.0,True,NaN,NaN,NaN,5,lv_rural1,NaN,MV1.101_LV1.103_eq,NaN
4,MV1.101 Load 5,7,2.986680e-17,1.719415e-13,0.0,0.0,0.0880,1.0,True,NaN,NaN,NaN,5,lv_rural1,NaN,MV1.101_LV1.104_eq,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,MV1.101 MV Load 1,15,2.755534e-20,6.102503e-15,0.0,0.0,0.3748,1.0,True,NaN,NaN,NaN,5,L2-M,NaN,MV1.101,NaN
92,MV1.101 MV Load 2,54,3.740617e-18,1.377551e-13,0.0,0.0,0.5998,1.0,True,NaN,NaN,NaN,5,G0-M,NaN,MV1.101,NaN
93,MV1.101 MV Load 3,64,8.510826e-13,5.331337e-24,0.0,0.0,0.4820,1.0,True,NaN,NaN,NaN,5,G3-M,NaN,MV1.101,NaN
94,MV1.101 MV Load 4,27,9.623625e-22,1.422984e-14,0.0,0.0,0.2891,1.0,True,NaN,NaN,NaN,5,G0-A,NaN,MV1.101,NaN
